<CENTER>
    <a href="http://opendata.atlas.cern/release/2020/documentation/notebooks/intro.html" class="icons"><img src="./images/ATLASOD.gif" style="width:40%"></a>
</CENTER>

<CENTER><h1>Searching for the Higgs boson </h1></CENTER>


Below are some Feyman diagrams of ways the Higgs boson can be produced according to the Standard Model:

<br>

<CENTER><img src="https://cds.cern.ch/record/2243593/files/Figures_FeynmanHprod.png" style="width:50%"></CENTER>

Just as the Higgs can be produced in several ways, the Higgs can also decay in a number of ways or 'channels'. In this excercise, we will be searching for the Higgs in two different decay channels.

---

<CENTER><h2>Search 1: The H&#8594;&gamma;&gamma; channel</h2></CENTER>

<CENTER><img src="./images/higgsFD.png" style="width:30%"></CENTER>

<br>
  
One of the ways the Higgs can decay is to two photons. We call this channel __H&#8594;&gamma;&gamma;__ ("Higgs to gamma gamma").


Of course, there are other ways two photons can be made in the LHC, but if we look at the entire range of invariant masses of these two photons, we should expect there to be more of them around 125 GeV, the mass of the Higgs ("bump hunting").

#### Importing the usual libraries

In [43]:
import ROOT
from ROOT import TMath
import time

### 1. Reading in ROOT files
<br>
In the usual way:

1. Open our datafile containing two photon ("diphoton" or $\gamma\gamma$) events 

2. Retrieive the TTree storing the data and 

3. Get the tree entries 

<br>

As always, if you need hints, refer back to the first tutorial `HistogramTutorial`. If you are really stuck, check out the solutions notebook `HiggsSearch_Solutions`

#### a) Open Root File

Using `ROOT`, `TFile.Open()` a sample of diphoton data, stored at 

`https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/GamGam/Data/data_A.GamGam.root`

 

In [44]:
#f = ####

#To solution:
f = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/GamGam/Data/data_A.GamGam.root")

#### b) Retrieve TTree

`Get()` the TTree named `mini` from the ROOT file `f`

In [45]:
#tree = ####

#To solution:
tree = f.Get("mini")

#### c) Load TTree data

`GetEntries()` of our `tree` to load in the data for each event stored in the tree

In [46]:
####

#To solution:
tree.GetEntries()

430344

### 2. Preparing histograms
<br>

As in previous examples, before we are ready to fill our histograms we need to:

1. Create a canvas for the histogram to be drawn onto

2. Define the settings of the histograms we intend to draw

#### a) Create canvas

Using `ROOT`, create a `TCanvas` named `"Canvas"` with the title `'cz'` that's sized `800`x`600` pixels

In [47]:
#canvas = ####

#To solution
canvas = ROOT.TCanvas("Canvas","cz",800,600)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: Canvas


#### b) Set up histogram 

Set up a `TH1F` type histogram to be filled with our __diphoton invariant masses__. Name it `h_M_Hyy`, give it the title `Diphoton invariant-mass`, label the x and y axes `Invariant Mass m_{yy} [GeV]` and `events` respectively and divide it in `30` bins between `105` and `160` GeV.

__Hint:__ Remember how we separate names, titles and axis labels in the `TH1F` function?

In [48]:
#Invariant mass histograms definition
#hist = ####

#To solution
hist = ROOT.TH1F("h_M_Hyy","Diphoton invariant-mass ; Invariant Mass m_{yy} [GeV] ; events",30,105,160)

### 3. Selecting events and filling histograms

Our strategy for filling our diphoton invariant mass histogram is as follows:

1. Loop through each `event` in our `tree`. We can print out how many events have been processed every 100000 events to keep track of progress.

2. In each event, search for _'good quality photons'_ (more on this later).

3. If there are exactly two qood quality photons, check that they are _'well isolated'_ (again, more later).

4. If the two photons are well-isolated, extract their 4 momentum from the components' individual TTree branches, converting their transverse momentum (pT) and energy (E) from MeV, as is stored in the TTree, to GeV as will be displayed in the histogram.

5. Add the TLorentz vectors of the two photons together

6. Fill the histogram with the invariant mass of our two-photon system

__NOTE:__ To simplify our code, we will be writing some custom __functions__ to handle some of the more complicated
operations performed in each event

#### a) Loop tracker

In [67]:
def trackProgress(n,m):
    """
    Function which prints the event loop progress every m events 
    
    Parameters
    ----------
    n : Number of events processed so far
    
    m : Printout event interval
    
    """
    print("Event loop tracker")
    print("------------------")
    if(n%m==0):
        print("%d events processed" % n)

#### b) Photon quality

In [50]:
#@NOTE: Explain 'triggers'

def locateGoodPhotons(tree):
    """
    Function which returns the index of photons in the event which pass our quality requirements.
    These are:
        - Event passes photon trigger
        - Photon is identified as such, passing 'Tight' requirements
        - Photon has pT > 25 GeV (or 25000 MeV)
        - Photon is in the 'central' region of ATLAS i.e. |eta| < 2.37
        - Photon does not fall in the 'transition region' between ATLAS's inner detector barrell
          and ECal encap i.e. 1.37 <= |eta| <= 1.52
          
    Parameters
    ----------
    tree : TTree entry for this event
    
    """
    
    ## Checking the event passes the photon trigger
    if(tree.trigP):
        
        #Initialise (set up) the variables we want to return
        goodphoton_index = [] #Indices (position in list of event's photons) of our good photons
            
        ##Loop through all the photons in the event
        for j in range(0,tree.photon_n):
            
            ##Check photon ID
            if(tree.photon_isTightID[j]):
                
                if(tree.photon_pt[j] > 25000 and  #Check photon has a large enough pT
                  (TMath.Abs(tree.photon_eta[j]) < 2.37) and  #Check photon eta is in the 'central' region
                  
                  #Exclude "transition region" between ID barrell and ECal endcap
                  (TMath.Abs(tree.photon_eta[j]) < 1.37 or TMath.Abs(tree.photon_eta[j]) > 1.52)):

                    goodphoton_index.append(j) #Store photon's index
                    
        return goodphoton_index #return list of good photon indices

#### c) Photon isolation

In [51]:
def photonIsolation(tree,photon_indices):
    """
    Function which returns True if all photons are well-isolated, otherwise returns false.
    
    A photon is considered 'isolated' if the transverse momentum and transverse energy within 
    a particular radius of the photon is below a certain threshold compared to the photon's 
    transverse momentum (don't worry too much about the details!).
    
    Parameters
    ----------
    tree : TTree entry for this event
    
    photon_indices : List containing the indices in the TTree of our photons of interest
    
    """
    
    #Loop through our list of photon indices
    for i in photon_indices:
        
        #If each photon passes isolation requirements...
        if((tree.photon_ptcone30[i]/tree.photon_pt[i] < 0.065) and 
           (tree.photon_etcone20[i] / tree.photon_pt[i] < 0.065)):
            continue #...keep the loop going 
        
        #If any fail, break the loop and return False
        else: 
            return False
    
    #If the loop is able to finish, i.e. all photons are well-isolated, return True
    return True

#### d) Extracting four-momentum

In [52]:
def photonFourMomentum(tree, photon_indices):
    """
    Function which returns the 4 momenta of a list of photons in an event as a list of TLorentzVectors
    
    Parameters
    ----------
    tree : TTree entry for this event
    
    photon_indices : List containing the indices in the TTree of our photons of interest
    
    """
    
    photon_four_momenta = []
    
    #Loop through our list of photon indices
    for i in photon_indices:
    
        #Initialse (set up) an empty 4 vector for each photon
        Photon_i = ROOT.TLorentzVector()
        
        #Retrieve the photon's 4 momentum components from the tree
        #Convert from MeV to GeV where needed
        Photon_i.SetPtEtaPhiE(tree.photon_pt[i]/1000., tree.photon_eta[i],tree.photon_phi[i],tree.photon_E[i]/1000.)
        
        #Store photon's 4 momentum
        photon_four_momenta.append(Photon_i)
        
        
    return photon_four_momenta

#### e) Sum the 4 momenta of each photon in the event

In [53]:
def sumFourMomentum(four_momenta):
    """
    Function which sums a list of four-momenta, and returns the resultant four-momentum of the system
    
    Parameters
    ----------
    four_momenta : List of TLorentzVectors containing the four-momentum of each object in the system
    
    """
    
    # Initialise (set up) TLorentzVector for our momentum sum
    four_mom_sum = ROOT.TLorentzVector()
    
    for obj in four_momenta:
        four_mom_sum += obj
        
    return four_mom_sum

### Putting it all together!

In [54]:
n = 0
for event in tree:
    n += 1
    
    #a) Loop progress tracking: Print progress every 100,000 events
    trackProgress(n,100000)
    
    #b) Identify exactly two 'good quality photons'
    goodphoton_indices = locateGoodPhotons(tree)
    
    if len(goodphoton_indices) == 2:
        
        #c) Check our good quality photons are well-isolated
        photons_are_isolated = photonIsolation(tree,goodphoton_indices)
        
        if photons_are_isolated:
        
            #d) Convert 4-momentum from MeV to GeV
            photon_four_momenta = photonFourMomentum(tree, goodphoton_indices)
            
            #e) Add the 4-momenta together
            Photon_12 = sumFourMomentum(photon_four_momenta)
            
            #f) Fill the histogram with the diphoton invariant mass
            inv_mass = Photon_12.M() #Calculated invariant mass
            
            hist.Fill(inv_mass) #Fill histogram with invariant mass

100000 events processed
200000 events processed
300000 events processed
400000 events processed


### 4. Draw plots

Finally, we would like to draw our diphoton invariant mass histograms and display the canvas showing our results. 

For this study, we would also like to plot the __error bars__ for each bin to illustrate the (statistical) __uncertainties__ on our measurement. This is done by passing `"E"` as an argument to `Draw()` when drawing the _histogram_.

In [55]:
#hist.####
#canvas.####

#To solution
hist.Draw("E")
canvas.Draw()

To make our plot easier to read, we can change the y axis from a __linear scale__ to a __log scale__. This is done by calling the `.SetLogY()` function on our `canvas`, then `Draw()`ing it agin

In [69]:
#hist.Draw("E")
#hist.SetMinimum(10)
canvas.SetLogy()
canvas.Draw()

### Some questions to think about...

1. Can we say we have 'found' the Higgs based on these histograms alone? Why/why not?
2. What steps could we take to make our search for the Higgs more robust?

---

<CENTER><h2>Search 2: The H&#8594;WW channel</h2></CENTER>

The following analysis is comparing the kinematics between events coming for the SM Higgs boson decaying to 2 W-bosons to those coming from the SM WW-diboson background  production.

SM Higgs to WW Feynman diagram:
<CENTER><img src="https://atlas.web.cern.ch/Atlas/GROUPS/PHYSICS/PAPERS/HIGG-2016-07/fig_01a.png" style="width:30%"></CENTER>

SM WW-diboson Feynman diagram:
<CENTER><img src="https://cds.cern.ch/record/1484203/files/fig1b.png" style="width:30%"></CENTER>

In [60]:
## reading the input files via internet (URL to the file)


## WW
bkg = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_363492.llvv.2lep.root")
t_bkg = bkg.Get("mini")
t_bkg.GetEntries()

3409043

In [61]:
## SM H->WW
sig = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_345324.ggH125_WW2lep.2lep.root")
t_sig = sig.Get("mini")
t_sig.GetEntries()


628685

In [62]:
c = ROOT.TCanvas("testCanvas","a first way to plot a variable",800,600)

In [63]:
h_bgs = ROOT.TH1F("h_bgs","Example plot: Missing transverse energey",20,0,200)
h2_bgs = ROOT.TH1F("h2_bgs","Example plot: Number of Jets",10,0,10)

h_sig = ROOT.TH1F("h_sig","Example plot: Missing transverse energey",20,0,200)
h2_sig = ROOT.TH1F("h2_sig","Example plot: Number of Jets",10,0,10)

In [64]:
n=0
for event in t_bkg:
    n += 1
    ## printing the evolution in number of events
    if(n%10000==0):
        print(n)
    h_bgs.Fill((t_bkg.met_et)/1000.)
    h2_bgs.Fill(t_bkg.jet_n)

m=0    
for event in t_sig:
    m += 1
    ## printing the evolution in number of events
    if(m%10000==0):
        print(m)
    h_sig.Fill((t_sig.met_et)/1000.)
    h2_sig.Fill(t_sig.jet_n)
        
print("Done!")

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

In [65]:
scale_bgs = h_bgs.Integral()
h_bgs.Scale(1/scale_bgs)

scale_sig = h_sig.Integral()
h_sig.Scale(1/scale_sig)


h_bgs.SetFillStyle(3001)
h_bgs.SetFillColor(4)
h_bgs.SetLineColor(4)

h_sig.SetFillStyle(3003)
h_sig.SetFillColor(2)
h_sig.SetLineColor(2)

legend=ROOT.TLegend(0.5,0.7,0.9,0.9)
legend.AddEntry(h_bgs,"Background (WW) ","l")
legend.AddEntry(h_sig,"Signal (H #rightarrow WW)","l")

h_sig.SetStats(0)
h_bgs.SetStats(0)

h_sig.Draw("hist")
h_bgs.Draw("histsame")
legend.Draw()
c.Draw()


In [66]:
scale2_bgs = h2_bgs.Integral()
h2_bgs.Scale(1/scale2_bgs)

scale2_sig = h2_sig.Integral()
h2_sig.Scale(1/scale2_sig)



h2_bgs.SetFillStyle(3001)
h2_bgs.SetFillColor(4)
h2_bgs.SetLineColor(4)

h2_sig.SetFillStyle(3003)
h2_sig.SetFillColor(2)
h2_sig.SetLineColor(2)

legend=ROOT.TLegend(0.5,0.7,0.9,0.9)
legend.AddEntry(h2_bgs,"Background (WW) ","l")
legend.AddEntry(h2_sig,"Signal (H #rightarrow WW)","l")


h2_sig.SetStats(0)
h2_bgs.SetStats(0)
h2_sig.Draw("hist")
h2_bgs.Draw("histsame")
legend.Draw()
c.Draw()
